In [1]:
#begin here

In [2]:
#import everything
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import math
import time
import keyboard
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
import spacy
import pydantic

In [3]:
print("spacy version:", spacy.__version__)
print("pydantic version:", pydantic.__version__)

spacy version: 3.7.5
pydantic version: 2.8.2
